In [243]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


###  importing necssary lib

In [244]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import OneHotEncoder
from sklearn.preprocessing  import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.tree import DecisionTreeClassifier

In [245]:
df=pd.read_csv("tested.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [246]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [247]:
df.isnull().sum()

Survived     0
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [248]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)
X_train.shape,X_test.shape,y_train.shape

((334, 7), (84, 7), (334,))

In [249]:
y_test.shape

(84,)

In [250]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [251]:
#filling missing value using imputer class 
si_sge=SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')
# fitting and transform train data and transform only on test data
X_train_age=si_sge.fit_transform(X_train[['Age']])
X_test_age=si_sge.transform(X_test[['Age']])
#embarked data
X_train_embarked=si_embarked.fit_transform(X_train[['Embarked']])
X_test_embarked=si_embarked.transform(X_test[['Embarked']])

In [252]:
# one hot encoding Sex and Embarked

ohe_sex = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [253]:
X_train_embarked.shape

(334, 3)

In [254]:
X_train_rem=X_train.drop(columns=['Age','Embarked','Sex'])

In [255]:
X_test_rem=X_test.drop(columns=['Age','Embarked','Sex'])

In [256]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)


In [257]:
X_test_transformed.shape

(84, 10)

In [258]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [259]:
y_pred=clf.predict(X_test_transformed)
y_pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1])

In [260]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0

In [261]:
print(X_train.shape, X_test.shape)


(334, 7) (84, 7)


In [262]:
print(X_train.columns)


Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')


In [263]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train_transformed, y_train, cv=5)
scores

array([1., 1., 1., 1., 1.])

In [264]:
import pickle
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))